In [ ]:
!pip install -r requirements.txt
!pip install numpy==1.26.4 scikit-learn==1.3.2 --force-reinstall --no-cache-dir


In [2]:
import os
import json
import random
import torch
import torch.nn as nn
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from peft import PeftModel, PeftConfig
from datasets import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from huggingface_hub import login

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
#==== CONFIGURATION ====
login(token="hf_...")
BGE_MODEL = "BAAI/bge-base-en-v1.5"
GEMMA_MODEL = "google/gemma-3-4b-pt"
CHECKPOINT_DIR = "./persistent_volume/last_checkpoint/"
BOOKS_PATH = "books.jsonl"
VAL_SPLIT = 0.005
BATCH_SIZE = 4
MAX_LENGTH = 256
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float32

# ==== LOAD MODELS ====
print("Loading tokenizer and base model...")
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_DIR)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    GEMMA_MODEL,
    torch_dtype=DTYPE,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, CHECKPOINT_DIR)
model.eval()
for p in model.parameters():
    p.requires_grad = False

print("Loading BGE encoder...")
bge_tokenizer = AutoTokenizer.from_pretrained(BGE_MODEL)
bge_encoder = AutoModel.from_pretrained(BGE_MODEL).to(DEVICE)
bge_encoder.eval()
for p in bge_encoder.parameters():
    p.requires_grad = False

# ==== PROJECTOR ====
class Projector(nn.Module):
    def __init__(self, in_dim=768, out_dim=2560):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, out_dim),
            nn.GELU(),
            nn.Linear(out_dim, out_dim)
        )

    def forward(self, x):
        return self.mlp(x)

projector = Projector().to(DEVICE)



In [4]:
# ==== DATA ====
print("Loading data...")
with open(BOOKS_PATH, "r", encoding="utf-8") as f:
    texts = [json.loads(line)["text"] for line in f if "text" in json.loads(line)]

random.shuffle(texts)
split_idx = int(len(texts) * (1 - VAL_SPLIT))
train_texts = texts[:split_idx]
val_texts = texts[split_idx:]

def chunk_text(text, max_tokens=MAX_LENGTH):
    tokens = tokenizer(text, truncation=False)["input_ids"]
    chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk) for chunk in chunks if len(chunk) > 10]

def encode_bge(texts):
    inputs = bge_tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
    with torch.no_grad():
        emb = bge_encoder(**inputs).last_hidden_state[:, 0, :]  # CLS token
    return emb

class ProjectorDataset(torch.utils.data.Dataset):
    def __init__(self, texts):
        self.samples = []
        for text in texts:
            for chunk in chunk_text(text):
                self.samples.append(chunk)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

def collate_fn(batch):
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LENGTH)
    input_ids = inputs["input_ids"][:, :-1].to(DEVICE)
    labels = inputs["input_ids"][:, 1:].to(DEVICE)
    attn_mask = inputs["attention_mask"][:, :-1].to(DEVICE)
    with torch.no_grad():
        bge_emb = encode_bge(batch)
    return bge_emb, input_ids, attn_mask, labels

train_dataset = ProjectorDataset(train_texts)
val_dataset = ProjectorDataset(val_texts)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
print("Data is ready")

Loading data...
Data is ready


In [ ]:

# ==== OPTIMIZER & LOSS ====
optimizer = torch.optim.AdamW(projector.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# ==== TRAIN LOOP ====
def evaluate():
    model.eval()
    projector.eval()
    total_loss, total_tokens = 0, 0
    with torch.no_grad():
        for bge_emb, input_ids, attn_mask, labels in val_loader:
            proj = projector(bge_emb)
            expanded_proj = proj.unsqueeze(1).expand(-1, input_ids.size(1), -1)
            inputs_embeds = model.base_model.model.language_model.embed_tokens(input_ids)
            full_inputs = inputs_embeds + expanded_proj
            full_inputs = full_inputs.to(dtype=DTYPE)
            output = model(inputs_embeds=full_inputs, attention_mask=attn_mask)
            logits = output.logits
            loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
            total_loss += loss.item() * labels.numel()
            total_tokens += labels.numel()
    ppl = torch.exp(torch.tensor(total_loss / total_tokens))
    return total_loss / total_tokens, ppl.item()

print("Starting training...")
for epoch in range(2):
    projector.train()
    for step, (bge_emb, input_ids, attn_mask, labels) in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()
        proj = projector(bge_emb)
        expanded_proj = proj.unsqueeze(1).expand(-1, input_ids.size(1), -1)
        inputs_embeds = model.base_model.model.language_model.embed_tokens(input_ids)
        full_inputs = inputs_embeds + expanded_proj
        full_inputs = full_inputs.to(dtype=DTYPE)
        output = model(inputs_embeds=full_inputs, attention_mask=attn_mask)
        logits = output.logits
        loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
        loss.backward()
        optimizer.step()
        if step % 25 == 0:
            val_loss, val_ppl = evaluate()
            print(f"[Epoch {epoch} | Step {step}] Train loss: {loss:.4f} | Val loss: {val_loss:.4f} | PPL: {val_ppl:.2f}")
            torch.save(projector.state_dict(), f"persistent_volume/proj_checkpoints/projector_step{step:05d}.pt")


# ==== SAVE ====  
torch.save(projector.state_dict(), "projector.pt")
print("Saved projector to projector.pt")


Starting training...


  0%|          | 1/80939 [01:37<2182:18:05, 97.07s/it]

[Epoch 0 | Step 0] Train loss: 3.9062 | Val loss: 3.1266 | PPL: 22.80


  0%|          | 26/80939 [03:25<660:26:38, 29.38s/it]

[Epoch 0 | Step 25] Train loss: 3.7812 | Val loss: 3.0252 | PPL: 20.60


...

  3%|▎         | 2301/80939 [2:53:44<673:48:59, 30.85s/it]

[Epoch 0 | Step 2300] Train loss: 4.3125 | Val loss: 5.7222 | PPL: 305.58


  3%|▎         | 2326/80939 [2:55:38<674:26:15, 30.89s/it]

[Epoch 0 | Step 2325] Train loss: 3.8594 | Val loss: 3.1935 | PPL: 24.37


  3%|▎         | 2350/80939 [2:55:50<11:16:22,  1.94it/s] 